In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

from imblearn.over_sampling import RandomOverSampler

import tensorflow as tf

In [2]:
df = pd.read_csv("diabetes.csv")
print(df.head())    

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [3]:
# plotting the data

'''
for i in range(len(df.columns[:-1])): # until the last column
    label = df.columns[i] # column labels
    plt.hist(df[df['Outcome']==1][label], color="blue", label="Diabetes", alpha=0.5, density=True) # positive
    plt.hist(df[df['Outcome']==0][label], color="red", label="No Diabetes", alpha=0.5, density=True) # negative
    plt.ylabel("Probability")
    plt.xlabel(label)
    plt.title(label)
    plt.legend()
    plt.show()
'''

'\nfor i in range(len(df.columns[:-1])): # until the last column\n    label = df.columns[i] # column labels\n    plt.hist(df[df[\'Outcome\']==1][label], color="blue", label="Diabetes", alpha=0.5, density=True) # positive\n    plt.hist(df[df[\'Outcome\']==0][label], color="red", label="No Diabetes", alpha=0.5, density=True) # negative\n    plt.ylabel("Probability")\n    plt.xlabel(label)\n    plt.title(label)\n    plt.legend()\n    plt.show()\n'

In [3]:
x_values = df[df.columns[:-1]].values # matrix
y_values = df[df.columns[-1]].values # output column

In [4]:
# scale the quantities (normalize the data)
scaler = StandardScaler()
x = scaler.fit_transform(x_values)

# make dataset balanced in terms of outcomes
over = RandomOverSampler()
x, y = over.fit_resample(x, y_values)

In [5]:
# 60% of the dataset for training, 20% for testing, 20% for validation
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=0) # training
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=0) # validation

In [6]:
# build model (layers, activation functions)
model = tf.keras.Sequential ([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # used for binary classification
])

# compile model (optimizer, loss function, metrics)
model.compile (
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [7]:
# train the model
model.fit(
    x_train, 
    y_train, 
    batch_size=2, 
    epochs=10,
    validation_data=(x_valid, y_valid)
)

Epoch 1/10
300/300 [==============================] - 1s 2ms/step - loss: 0.5673 - accuracy: 0.7017 - val_loss: 0.4658 - val_accuracy: 0.7550
Epoch 2/10
300/300 [==============================] - 0s 1ms/step - loss: 0.5008 - accuracy: 0.7517 - val_loss: 0.4760 - val_accuracy: 0.7750
Epoch 3/10
300/300 [==============================] - 0s 1ms/step - loss: 0.4884 - accuracy: 0.7517 - val_loss: 0.4541 - val_accuracy: 0.7750
Epoch 4/10
300/300 [==============================] - 0s 1ms/step - loss: 0.4684 - accuracy: 0.7833 - val_loss: 0.4784 - val_accuracy: 0.7450
Epoch 5/10
300/300 [==============================] - 0s 1ms/step - loss: 0.4594 - accuracy: 0.7833 - val_loss: 0.4583 - val_accuracy: 0.7800
Epoch 6/10
300/300 [==============================] - 0s 1ms/step - loss: 0.4442 - accuracy: 0.7883 - val_loss: 0.4520 - val_accuracy: 0.7600
Epoch 7/10
300/300 [==============================] - 0s 1ms/step - loss: 0.4281 - accuracy: 0.7883 - val_loss: 0.4496 - val_accuracy: 0.7750
Epoch 

In [8]:
test = []

with open('test.csv') as file:
    output = [float(s) for line in file.readlines() for s in line[:-1].split(',')]
    test.append(output);

print(test)

[[1.0, 93.0, 70.0, 31.0, 0.0, 30.4, 0.315, 2.0]]


In [10]:
test = np.array(test).reshape(-1, 1)
normalized_array = scaler.fit_transform(test)

if(model.predict(np.array([normalized_array])) > 0.5):
    print("Diabetes")
else:
    print("No diabetes")

1/1 [==============================] - 0s 15ms/step
No diabetes
